In [1]:
using BridgeStan
using Pigeons
using Random

struct StanCtDNAModel end

# initialize Stan model with ctdna data
function init_stan_ctdna(n, num_clones, clone_cn_profiles, ctdna, scale)
    stan_file = "model/simple_model.stan"
    stan_data = Pigeons.json(; n=n, num_clones=num_clones, clone_cn_profiles=clone_cn_profiles, ctdna=ctdna, scale=scale)

    return StanLogPotential(stan_file, stan_data, StanCtDNAModel())
end


init_stan_ctdna (generic function with 1 method)

In [6]:
dump(Pigeons.stan_funnel(2))

StanLogPotential{...}
  model: StanModel
    lib: Ptr{Nothing} @0x0000000001ea4dd0
    stanmodel: Ptr{BridgeStan.StanModelStruct} @0x0000000001faf1d0
    data: String "{\"dim\" : 2,\"scale\" : 2.0}"
    seed: UInt32 0x000000cc
  stan_file: String "/home/matthew/.julia/packages/Pigeons/UUYm9/examples/stan/funnel.stan"
  data: Pigeons.Immutable{String}
    hash: UInt64 0xe23920bf297290ac
    data: String "{\"dim\" : 2,\"scale\" : 2.0}"
  extra_information: Nothing nothing


┌ Warning: Loading a shared object '/home/matthew/.julia/packages/Pigeons/UUYm9/examples/stan/funnel_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan ~/.julia/packages/BridgeStan/aKsXp/src/model.jl:58


In [8]:
methods(Pigeons.toy_turing_unid_target)


# 0 methods for generic function "toy_turing_unid_target" from Pigeons

In [2]:
#read in data
using CSV
using DataFrames

ctdna_path = "data/ctdna.tsv"
clones_path = "data/2-clones-simple.tsv"

ctdna_data = CSV.read(ctdna_path, DataFrame, delim='\t',header=false)
clones_data = CSV.read(clones_path, DataFrame, delim='\t')
# check the first few rows
println(first(ctdna_data, 5))
println(first(clones_data,5))


5×1 DataFrame
 Row │ Column1  
     │ Float64  
─────┼──────────
   1 │ 0.404146
   2 │ 0.394167
   3 │ 0.392297
   4 │ 0.385272
   5 │ 0.39825
5×3 DataFrame
 Row │ bin_id   Clone_0  Clone_1 
     │ String7  Int64    Int64   
─────┼───────────────────────────
   1 │ bin_0         10        1
   2 │ bin_1         10        1
   3 │ bin_2         10        1
   4 │ bin_3         10        1
   5 │ bin_4         10        1


In [3]:
n = size(clones_data)[1]
num_clones = size(clones_data)[2]-1
scale = 1.0 #for simplicity
# clones_data = Matrix(clones_data[:,2:3])
# ctdna_data = Vector(ctdna_data[:,1])
# pt = pigeons(target=init_stan_ctdna(n, num_clones, clones_data, ctdna_data, scale))

# Note: Depending on the application, you might need a different setup for the reference model.


1.0

In [4]:
using CSV
using DataFrames
using JSON
data_dict = Dict(
    "n" => n,
    "num_clones" => num_clones,
    "clone_cn_profiles" => transpose(Matrix(clones_data[:, 2:end])),  # Transpose to fit matrix[n, num_clones]
    "ctdna" => Vector(ctdna_data[:,1]),  # This assumes you've added a 'ctdna' column to your DataFrame
    "scale" => 1.0
)
json_data = JSON.json(data_dict)
print(json_data)

{"ctdna":[0.40414565801620483,0.39416712522506714,0.3922972083091736,0.3852720260620117,0.3982504606246948,0.41510510444641113,0.40069669485092163,0.40178537368774414,0.40535756945610046,0.3901507556438446,0.35494464635849,0.3953408896923065,0.4033311605453491,0.3931475579738617,0.37400615215301514,0.38480669260025024,0.39034488797187805,0.39247381687164307,0.40081900358200073,0.3932713568210602,0.39404648542404175,0.4087708592414856,0.3775710165500641,0.39758992195129395,0.37430280447006226,-0.6490795612335205,-0.6484686136245728,-0.6511326432228088,-0.6476861238479614,-0.6362817883491516,-0.6555114388465881,-0.6334537267684937,-0.6413617730140686,-0.6604368686676025,-0.6473173499107361,-0.6457176804542542,-0.6619637608528137,-0.6493025422096252,-0.6352083086967468,-0.6713491678237915,-0.5998506546020508,-0.6640127897262573,-0.6458156704902649,-0.6748106479644775,-0.6597251892089844,-0.6675729751586914,-0.6498703956604004,-0.6642107963562012,-0.6618372201919556,-0.6529645323753357],"n

In [5]:
stan_file = "model/simple_model.stan"
StanLogPotential(stan_file,json_data)

StanLogPotential(simple_model_model)

In [6]:
pt = pigeons(target = StanLogPotential(stan_file,json_data), record = [traces; record_default()])

┌ Warning: Loading a shared object '/home/matthew/laopo/liquid_pigeon/model/simple_model_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan ~/.julia/packages/BridgeStan/aKsXp/src/model.jl:58
┌ Warning: It looks like sample_iid!() is not implemented for a 
│ reference_log_potential of type StanLogPotential{...}. 
│ Instead, using step!(). 
└ @ Pigeons ~/.julia/packages/Pigeons/UUYm9/src/targets/target.jl:51


──────────────────────────────────────────────────────────────────────────────────────────────────
  scans        Λ        time(s)    allc(B)  log(Z₁/Z₀)   min(α)     mean(α)    min(αₑ)   mean(αₑ) 
────────── ────────── ────────── ────────── ────────── ────────── ────────── ────────── ──────────
        2   7.11e-15      0.326   1.98e+07   1.78e-14          1          1          0      0.456 
        4   1.07e-14     0.0385   5.57e+04   3.55e-15          1          1      0.351      0.613 
        8   1.07e-14     0.0896   1.06e+05   8.88e-15          1          1      0.394      0.608 
       16   1.42e-14      0.168   2.07e+05   1.11e-15          1          1       0.54      0.654 
       32    1.2e-14      0.368   2.68e+05  -2.22e-16          1          1      0.531      0.598 
       64   1.54e-14      0.696   3.74e+05  -1.78e-15          1          1      0.593      0.663 
      128   1.31e-14       1.47   6.25e+05   1.33e-15          1          1       0.57      0.623 
      256 

PT(checkpoint = false, ...)

In [7]:
using InferenceReport

report(pt)

target_description...[skipped: no description provided]
pair_plot...

┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QGv02/src/PairPlots.jl:1115
┌ Warning: Could not compute valid contours
└ @ PairPlots ~/.julia/packages/PairPlots/QG

[skipped: InexactError(:trunc, Int64, -Inf)]
trace_plot...[skipped: InterruptException()]
moments...

In [4]:
using BridgeStan
using Pigeons
using Random

# We will use this type to make sure our iid sampler (next section) will
# be used only for this model
struct StanUnidentifiableExample end

function stan_unid(n_trials, n_successes)
    # path to a .stan file (compiled files will be cached in the same directory)
    stan_file = dirname(dirname(pathof(Pigeons))) * "/examples/stan/unid.stan"

    # data can be specified either using...
    #   - a path to a json file with suffix .json containing the data to condition on
    #   - the JSON string itself (here via the utility Pigeons.json())
    stan_data = Pigeons.json(; n_trials, n_successes)

    return StanLogPotential(stan_file, stan_data, StanUnidentifiableExample())
end

pt = pigeons(target = stan_unid(100, 50), reference = stan_unid(0, 0),record = [traces; record_default()])


┌ Warning: Loading a shared object '/home/matthew/.julia/packages/Pigeons/UUYm9/examples/stan/unid_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan ~/.julia/packages/BridgeStan/aKsXp/src/model.jl:58
┌ Warning: Loading a shared object '/home/matthew/.julia/packages/Pigeons/UUYm9/examples/stan/unid_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan ~/.julia/packages/BridgeStan/aKsXp/src/model.jl:58


──────────────────────────────────────────────────────────────────────────────────────────────────
  scans        Λ        time(s)    allc(B)  log(Z₁/Z₀)   min(α)     mean(α)    min(αₑ)   mean(αₑ) 
────────── ────────── ────────── ────────── ────────── ────────── ────────── ────────── ──────────
        2        1.3    0.00169   3.82e+04      -5.89     0.0027      0.856      0.351      0.569 
        4       1.85    0.00443    4.6e+04      -4.38      0.592      0.794      0.369      0.585 
        8       1.36    0.00858   7.79e+04      -4.83      0.282      0.849      0.519      0.618 
       16       1.56     0.0177   1.39e+05      -4.88      0.595      0.827      0.503      0.586 
       32       1.49     0.0304   2.14e+05      -4.55       0.67      0.835      0.568      0.654 
       64        1.4     0.0607   2.73e+05      -4.84      0.695      0.844       0.57      0.628 
      128       1.53      0.123   4.13e+05      -4.95      0.754       0.83      0.552      0.618 
      256 

PT(checkpoint = false, ...)

In [5]:
using InferenceReport
report(pt)

target_description...[skipped: no description provided]
pair_plot... ✓
trace_plot... ✓
moments... ✓
trace_plot_cumulative... ✓
mpi_standard_out...[skipped: no MPI std out files found]
lcb... ✓
gcb_progress... ✓
logz_progress... ✓
round_trip_progress...[skipped: number restarts not recorded]
pigeons_summary... ✓
pigeons_inputs... ✓
reproducibility_info...[skipped: missing reproducibility_command]


┌ Warning: Unable to determine HTML(edit_link = ...) from remote HEAD branch, defaulting to "master".
│ Calling `git remote` failed with an exception. Set JULIA_DEBUG=Documenter to see the error.
│ Unless this is due to a configuration error, the relevant variable should be set explicitly.
└ @ Documenter ~/.julia/packages/Documenter/CJeWX/src/utilities/utilities.jl:640
[ Info: SetupBuildDirectory: setting up build directory.
[ Info: Doctest: running doctests.
[ Info: ExpandTemplates: expanding markdown templates.
[ Info: CrossReferences: building cross-references.
[ Info: CheckDocument: running document checks.
[ Info: Populate: populating indices.
[ Info: RenderDocument: rendering document.
[ Info: HTMLWriter: rendering HTML pages.
┌ Warning: Unable to determine the repository root URL for the navbar link.
│ This can happen when a string is passed to the `repo` keyword of `makedocs`.
│ 
│ To remove this warning, either pass a Remotes.Remote object to `repo` to completely
│ specify the

Report generated at: /home/matthew/laopo/liquid_pigeon/results/all/2024-05-10-15-08-29-gJw7jy2L


┌ Warning: Cannot extract version for inventory from /home/matthew/laopo/liquid_pigeon/results/all/Project.toml: no such file
└ @ Documenter.HTMLWriter ~/.julia/packages/Documenter/CJeWX/src/html/write_inventory.jl:89
┌ Warning: Please set `inventory_version` in the `HTML()` options passed to `makedocs`.
└ @ Documenter.HTMLWriter ~/.julia/packages/Documenter/CJeWX/src/html/write_inventory.jl:98
┌ Warning: Unable to find `xdg-open`. Try `apt install xdg-open`
└ @ InferenceReport ~/.julia/packages/InferenceReport/FRdM0/src/utils.jl:21
